# Healthcare Data Preprocessing and Integration

This notebook demonstrates the preprocessing and integration of healthcare data from various sources, including patient records, ICD-10 codes, and medical images, to prepare a unified dataset for downstream tasks like clinical note generation.

# Module 3: Healthcare Data Preprocessing for Clinical Note Generation
In this step, we will:
1. Load the healthcare dataset.
2. Drop irrelevant administrative columns.
3. Convert numeric and date columns into descriptive text.
4. Prepare a readable dataset for further text-based AI processing.


In [ ]:
import pandas as pd

# Path to your dataset (use the one from Module 1 or the initial healthcare dataset)
healthcare_path = "/content/healthcare_dataset.csv"
df = pd.read_csv(healthcare_path)

print("✅ Healthcare dataset loaded successfully!")
print("Shape:", df.shape)
display(df.head())


✅ Healthcare dataset loaded successfully!
Shape: (55500, 15)


,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


## Step 1: Select Relevant Columns
We'll keep only the columns useful for clinical summarization and note generation.
These include:
- Patient identifiers (Name, Age, Gender)
- Medical details (Blood Type, Medical Condition, Medication, Test Results)
- Context (Doctor, Hospital, Admission Type)
- Timeline (Date of Admission, Discharge Date)


In [ ]:
# Keep only relevant columns
keep_cols = [
    "Name", "Age", "Gender", "Blood Type", "Medical Condition",
    "Date of Admission", "Discharge Date", "Doctor", "Hospital",
    "Admission Type", "Medication", "Test Results"
]

df = df[keep_cols].copy()
print("✅ Selected columns:")
print(df.columns.tolist())
display(df.head(3))


✅ Selected columns:
['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition', 'Date of Admission', 'Discharge Date', 'Doctor', 'Hospital', 'Admission Type', 'Medication', 'Test Results']


,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,2024-02-02,Matthew Smith,Sons and Miller,Urgent,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,2019-08-26,Samantha Davies,Kim Inc,Emergency,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,2022-10-07,Tiffany Mitchell,Cook PLC,Emergency,Aspirin,Normal


## Step 2: Convert Dates, Numbers, and Categories into Text
- Dates → “January 31, 2024”  
- Age → “30-year-old”  
- Blood Type → embedded in the description  
- Admission Type → standardized text (urgent, routine, emergency, etc.)


In [37]:
from datetime import datetime
import pandas as pd
import uuid

def format_date(date_str):
    """Convert date string (DD-MM-YYYY or similar) into readable Month DD, YYYY"""
    try:
        return datetime.strptime(str(date_str), "%d-%m-%Y").strftime("%B %d, %Y")
    except:
        try:
            return datetime.strptime(str(date_str), "%Y-%m-%d").strftime("%B %d, %Y")
        except:
            return str(date_str)

df["Date of Admission"] = df["Date of Admission"].apply(format_date)
df["Discharge Date"] = df["Discharge Date"].apply(format_date)

# Age to text - only apply if the column is not already in the desired text format
if not df['Age'].astype(str).str.contains('-year-old').any():
    df["Age"] = df["Age"].apply(lambda x: f"{int(x)}-year-old" if pd.notnull(x) else "age not specified")


# Standardize Admission Type text
df["Admission Type"] = df["Admission Type"].str.capitalize().replace({
    "Urgent": "urgent admission",
    "Emergency": "emergency admission",
    "Routine": "routine admission",
    "Elective": "elective admission"
})

# Explicitly drop the 'Age_Text' column if it exists
if 'Age_Text' in df.columns:
    df = df.drop(columns=['Age_Text'])

# Add a unique patient ID
if 'patient_id' not in df.columns:
    df.insert(0, 'patient_id', range(1001, 1001 + len(df)))


print("✅ Dates, age, and admission type converted to text.")
display(df.head(3))

✅ Dates, age, and admission type converted to text.


,patient_id,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results
0,1,Bobby Jackson,30-year-old,Male,B,Cancer,"January 31, 2024","February 02, 2024",Matthew Smith,Sons And Miller,Urgent admission,Paracetamol,Normal
1,2,Leslie Terry,62-year-old,Male,A+,Obesity,"August 20, 2019","August 26, 2019",Samantha Davies,Kim Inc,Emergency admission,Ibuprofen,Inconclusive
2,3,Danny Smith,76-year-old,Female,A,Obesity,"September 22, 2022","October 07, 2022",Tiffany Mitchell,Cook Plc,Emergency admission,Aspirin,Normal


## Step 3: Standardize Text Columns
Make textual columns uniform (capitalize names, remove special symbols, etc.)


In [36]:
# Basic cleaning
for col in ["Name", "Gender", "Blood Type", "Medical Condition", "Doctor", "Hospital", "Medication", "Test Results"]:
    df[col] = df[col].astype(str).str.strip().str.replace(r"[_-]+", " ", regex=True).str.title()

print("✅ Cleaned textual columns.")
display(df.head(5))

✅ Cleaned textual columns.


,patient_id,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results
0,1,Bobby Jackson,30-year-old,Male,B,Cancer,"January 31, 2024","February 02, 2024",Matthew Smith,Sons And Miller,Urgent admission,Paracetamol,Normal
1,2,Leslie Terry,62-year-old,Male,A+,Obesity,"August 20, 2019","August 26, 2019",Samantha Davies,Kim Inc,Emergency admission,Ibuprofen,Inconclusive
2,3,Danny Smith,76-year-old,Female,A,Obesity,"September 22, 2022","October 07, 2022",Tiffany Mitchell,Cook Plc,Emergency admission,Aspirin,Normal
3,4,Andrew Watts,28-year-old,Female,O+,Diabetes,"November 18, 2020","December 18, 2020",Kevin Wells,"Hernandez Rogers And Vang,",Elective admission,Ibuprofen,Abnormal
4,5,Adrienne Bell,43-year-old,Female,Ab+,Cancer,"September 19, 2022","October 09, 2022",Kathleen Hanna,White White,Urgent admission,Penicillin,Abnormal


In [39]:
import pandas as pd

# Load your preprocessed healthcare dataset
# df = pd.read_csv("/content/cleaned_data/healthcare_preprocessed.csv") # Removed this line

# Optional: If patient_id not present, create it
if 'patient_id' not in df.columns:
    df.insert(0, 'patient_id', range(1001, 1001 + len(df)))

# Create a clean, readable textual input for the model
df['summary_input'] = (
    "Patient " + df['Name'] + ", a " + df['Age'] + " " + df['Gender'].str.lower() +
    " with blood type " + df['Blood Type'] +
    ", was admitted on " + df['Date of Admission'] +
    " for " + df['Medical Condition'] + ". " +
    "The admission type was " + df['Admission Type'] +
    " under Dr. " + df['Doctor'] + " at " + df['Hospital'] + ". " +
    "Medication prescribed: " + df['Medication'] +
    ". Test Results: " + df['Test Results'] + "."
)

# Display first few examples
display(df[['patient_id', 'summary_input']].head())

,patient_id,summary_input
0,1,"Patient Bobby Jackson, a 30-year-old male with..."
1,2,"Patient Leslie Terry, a 62-year-old male with ..."
2,3,"Patient Danny Smith, a 76-year-old female with..."
3,4,"Patient Andrew Watts, a 28-year-old female wit..."
4,5,"Patient Adrienne Bell, a 43-year-old female wi..."


In [44]:
import pandas as pd

# Load your preprocessed healthcare data
df = pd.read_csv("/content/healthcare_dataset_preprocessed.csv")

# Create new patient IDs in proper hospital-like format
df.insert(0, 'patient_id', ['P' + str(1000 + i) for i in range(len(df))])

# Create condition keywords for later ICD mapping
df['condition_keywords'] = df['Medical Condition'].str.lower().str.strip()

# Create improved textual summaries for GenAI input
df['summary_input'] = (
    "Patient " + df['Name'] + ", a " + df['Age'] + " " + df['Gender'].str.lower() +
    " with blood type " + df['Blood Type'] +
    ", was admitted on " + df['Date of Admission'] +
    " for " + df['Medical Condition'] + ". " +
    "The admission was " + df['Admission Type'].str.lower() +
    " under Dr. " + df['Doctor'] + " at " + df['Hospital'] + ". " +
    "Medication prescribed includes " + df['Medication'] +
    ". Test results were " + df['Test Results'] + "."
)

# Display a few rows
df[['patient_id', 'summary_input', 'condition_keywords']].head()


,patient_id,summary_input,condition_keywords
0,P1000,"Patient Bobby Jackson, a 30-year-old male with...",cancer
1,P1001,"Patient Leslie Terry, a 62-year-old male with ...",obesity
2,P1002,"Patient Danny Smith, a 76-year-old female with...",obesity
3,P1003,"Patient Andrew Watts, a 28-year-old female wit...",diabetes
4,P1004,"Patient Adrienne Bell, a 43-year-old female wi...",cancer


In [45]:
df.to_csv("/content/cleaned_data/healthcare_textual_ready.csv", index=False)
print("✅ Healthcare textual dataset saved successfully!")


✅ Healthcare textual dataset saved successfully!


In [46]:
from google.colab import files

# Assuming the final dataset is saved at this path
files.download('/content/cleaned_data/healthcare_textual_ready.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Step 4  — Integrate All Processed Data Sources  

In this step, we’ll merge:
- Preprocessed Healthcare Dataset (EHR)
- ICD-10 Dataset (code + description)
- Enhanced Image Index (X-ray + Prescription from Module 2)

Goal:
Create one unified dataset (`merged_patient_dataset.csv`) that contains all relevant info for each patient record — ready for text generation using an LLM.


In [50]:
import pandas as pd
import numpy as np
import os

# Define paths to all datasets
healthcare_path = "/content/cleaned_data/healthcare_textual_ready.csv"       # from your preprocessed step
icd_path = "/content/cleaned_icdcodeset.csv"
enhanced_index_path = "/content/enhanced_image_index.csv"

# Load datasets
df_health = pd.read_csv(healthcare_path)
df_icd = pd.read_csv(icd_path)
df_imgs = pd.read_csv(enhanced_index_path)

print("✅ Datasets Loaded Successfully!")
print("Healthcare:", df_health.shape)
print("ICD Codes:", df_icd.shape)
print("Enhanced Image Index:", df_imgs.shape)

display(df_health.head(2))
display(df_icd.head(2))
display(df_imgs.head(2))


✅ Datasets Loaded Successfully!
Healthcare: (55500, 15)
ICD Codes: (73639, 2)
Enhanced Image Index: (5680, 4)


,patient_id,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results,condition_keywords,summary_input
0,P1000,Bobby Jackson,30-year-old,Male,B,Cancer,"January 31, 2024","February 02, 2024",Matthew Smith,Sons And Miller,Urgent admission,Paracetamol,Normal,cancer,"Patient Bobby Jackson, a 30-year-old male with..."
1,P1001,Leslie Terry,62-year-old,Male,A+,Obesity,"August 20, 2019","August 26, 2019",Samantha Davies,Kim Inc,Emergency admission,Ibuprofen,Inconclusive,obesity,"Patient Leslie Terry, a 62-year-old male with ..."


,ICDCode,Description
0,A000,"Cholera due to Vibrio cholerae 01, biovar c..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar e..."


,original_path,enhanced_path,psnr,ssim
0,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000115.png,21.593953,0.746678
1,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000117 ...,23.827213,0.705685


In [49]:
!unzip /content/prescription_enhanced.zip -d /content/
!unzip /content/xray_enhanced.zip -d /content/

Archive:  /content/prescription_enhanced.zip
replace /content/prescription_enhanced/0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/prescription_enhanced/0.png  
  inflating: /content/prescription_enhanced/1.png  
  inflating: /content/prescription_enhanced/10.png  
  inflating: /content/prescription_enhanced/100.png  
  inflating: /content/prescription_enhanced/1000.png  
  inflating: /content/prescription_enhanced/1001.png  
  inflating: /content/prescription_enhanced/1002.png  
  inflating: /content/prescription_enhanced/1003.png  
  inflating: /content/prescription_enhanced/1004.png  
  inflating: /content/prescription_enhanced/1005.png  
  inflating: /content/prescription_enhanced/1006.png  
  inflating: /content/prescription_enhanced/1007.png  
  inflating: /content/prescription_enhanced/1008.png  
  inflating: /content/prescription_enhanced/1009.png  
  inflating: /content/prescription_enhanced/101.png  
  inflating: /content/prescription_enhanced/1010.png  

### Step 5 — Clean & Prepare the ICD and Image Index Datasets

We’ll:
- Keep only relevant columns
- Clean column names
- Ensure patient IDs, X-ray IDs, and prescription IDs match a uniform format


In [53]:
# Clean ICD dataset
df_icd.columns = df_icd.columns.str.strip().str.replace(" ", "_")
df_icd = df_icd.dropna().drop_duplicates()

# Clean image index dataset
df_imgs.columns = df_imgs.columns.str.strip().str.replace(" ", "_")
df_imgs = df_imgs[df_imgs['enhanced_path'].notna()]

# Just to ensure unique patient mapping
# df_imgs = df_imgs.drop_duplicates(subset=['xray_id', 'prescription_id'], keep='first') # Removed this line

print("✅ ICD & Image Data Cleaned")
display(df_icd.head())
display(df_imgs.head())

✅ ICD & Image Data Cleaned


,ICDCode,Description
0,A000,"Cholera due to Vibrio cholerae 01, biovar c..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar e..."
2,A009,"Cholera, unspecified"
3,A0100,"Typhoid fever, unspecified"
4,A0101,Typhoid meningitis


,original_path,enhanced_path,psnr,ssim
0,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000115.png,21.593953,0.746678
1,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000117 ...,23.827213,0.705685
2,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000114.png,21.140863,0.734485
3,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000112 ...,20.423558,0.687321
4,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000111 ...,22.170544,0.720717


### Step 6 — Merge Datasets Logically

We’ll merge:
1. **Healthcare data** → Base dataset  
2. **ICD-10 data** → Mapped by Medical Condition (simple mapping for now)  
3. **Enhanced image index** → Linked by patient_id / IDs  

Later in real integration, semantic ICD mapping and embedding similarity matching can replace this simple join.


In [56]:
# --- Quick ICD assignment to save time ---
df_health["ICD_Code"] = np.random.choice(df_icd["ICDCode"], size=len(df_health))
df_health = df_health.merge(df_icd, left_on="ICD_Code", right_on="ICDCode", how="left")
df_health.rename(columns={"Description": "ICD_Description"}, inplace=True)
df_health.drop(columns=["ICDCode"], inplace=True)

print("✅ ICD mapping completed (random fast assignment for now)")
display(df_health.sample(1))

✅ ICD mapping completed (random fast assignment for now)


,patient_id,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results,condition_keywords,summary_input,ICD_Code,ICD_Description,ICD_Description
43898,P44898,Julie Larson,62-year-old,Male,O,Arthritis,"October 01, 2023","October 25, 2023",Kathleen Martinez,Taylor Barrett,Emergency admission,Paracetamol,Normal,arthritis,"Patient Julie Larson, a 62-year-old male with ...",T2044XA,"War operations involving biological weapons, c...",Corrosion of unspecified degree of nose (septu...


In [58]:
# --- Fix ICD Mapping (no duplicates) ---
df_health.drop(columns=[col for col in df_health.columns if "ICD_Description" in col], inplace=True, errors='ignore')

# Random ICD code assignment (fast)
df_health["ICD_Code"] = np.random.choice(df_icd["ICDCode"], size=len(df_health))

# Merge only the required ICD description
df_health = df_health.merge(df_icd[["ICDCode", "Description"]], left_on="ICD_Code", right_on="ICDCode", how="left")

# Rename cleanly
df_health.rename(columns={"Description": "ICD_Description"}, inplace=True)
df_health.drop(columns=["ICDCode"], inplace=True)

print("✅ ICD mapping completed (clean, single ICD_Description column).")
display(df_health.head(3))

# Save the dataframe
df_health.to_csv("/content/cleaned_data/healthcare_with_icd.csv", index=False)
print("✅ Healthcare dataset with ICD mapping saved successfully!")

✅ ICD mapping completed (clean, single ICD_Description column).


,patient_id,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results,condition_keywords,summary_input,ICD_Code,ICD_Description
0,P1000,Bobby Jackson,30-year-old,Male,B,Cancer,"January 31, 2024","February 02, 2024",Matthew Smith,Sons And Miller,Urgent admission,Paracetamol,Normal,cancer,"Patient Bobby Jackson, a 30-year-old male with...",S80279A,"Other superficial bite of unspecified knee, in..."
1,P1001,Leslie Terry,62-year-old,Male,A+,Obesity,"August 20, 2019","August 26, 2019",Samantha Davies,Kim Inc,Emergency admission,Ibuprofen,Inconclusive,obesity,"Patient Leslie Terry, a 62-year-old male with ...",T17298D,Other foreign object in pharynx causing other ...
2,P1002,Danny Smith,76-year-old,Female,A,Obesity,"September 22, 2022","October 07, 2022",Tiffany Mitchell,Cook Plc,Emergency admission,Aspirin,Normal,obesity,"Patient Danny Smith, a 76-year-old female with...",L234,Allergic contact dermatitis due to dyes


✅ Healthcare dataset with ICD mapping saved successfully!


In [61]:
# =====================================================================
# IMAGE LINKING (for Clinical Note Generation)
# =====================================================================

import pandas as pd
import numpy as np
import os

# ---------------------------------------------------------------------
# 1️⃣ LOAD PROCESSED DATASETS
# ---------------------------------------------------------------------
healthcare_path = "/content/cleaned_data/healthcare_with_icd.csv"      # after ICD merge
image_index_path = "/content/enhanced_image_index.csv"    # from Module 2

df_health = pd.read_csv(healthcare_path)
df_images = pd.read_csv(image_index_path)

print("Healthcare + ICD dataset:", df_health.shape)
print("Enhanced image index:", df_images.shape)

display(df_health.head(2))
display(df_images.head(2))

# ---------------------------------------------------------------------
# 2️⃣ SPLIT IMAGE TYPES
# ---------------------------------------------------------------------
# Infer image type from original_path
df_xray = df_images[df_images["original_path"].str.lower().str.contains("xray", na=False)]
df_pres = df_images[df_images["original_path"].str.lower().str.contains("prescription", na=False)]


print(f"Found {len(df_xray)} X-ray images, {len(df_pres)} prescription images.")

# ---------------------------------------------------------------------
# 3️⃣ SYNTHETICALLY LINK IMAGES TO EACH PATIENT
# ---------------------------------------------------------------------
np.random.seed(42)  # for reproducibility

df_health["xray_image"] = np.random.choice(df_xray["enhanced_path"], size=len(df_health), replace=True)
df_health["prescription_image"] = np.random.choice(df_pres["enhanced_path"], size=len(df_health), replace=True)

print("✅ Linked synthetic image paths to each patient record.")

display(df_health[["patient_id", "Medical Condition", "xray_image", "prescription_image"]].head(5))

# ---------------------------------------------------------------------
# 4️⃣ SAVE THE FINAL COMBINED DATASET
# ---------------------------------------------------------------------
output_path = "/content/cleaned_data/healthcare_with_images.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_health.to_csv(output_path, index=False)

print(f"💾 Saved merged dataset → {output_path}")

Healthcare + ICD dataset: (55500, 17)
Enhanced image index: (5680, 4)


,patient_id,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Discharge Date,Doctor,Hospital,Admission Type,Medication,Test Results,condition_keywords,summary_input,ICD_Code,ICD_Description
0,P1000,Bobby Jackson,30-year-old,Male,B,Cancer,"January 31, 2024","February 02, 2024",Matthew Smith,Sons And Miller,Urgent admission,Paracetamol,Normal,cancer,"Patient Bobby Jackson, a 30-year-old male with...",S80279A,"Other superficial bite of unspecified knee, in..."
1,P1001,Leslie Terry,62-year-old,Male,A+,Obesity,"August 20, 2019","August 26, 2019",Samantha Davies,Kim Inc,Emergency admission,Ibuprofen,Inconclusive,obesity,"Patient Leslie Terry, a 62-year-old male with ...",T17298D,Other foreign object in pharynx causing other ...


,original_path,enhanced_path,psnr,ssim
0,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000115.png,21.593953,0.746678
1,/content/xray_data/XraysData/valid/adenocarcin...,/content/enhanced_images/xray_enhanced/000117 ...,23.827213,0.705685


Found 1000 X-ray images, 4680 prescription images.
✅ Linked synthetic image paths to each patient record.


,patient_id,Medical Condition,xray_image,prescription_image
0,P1000,Cancer,/content/enhanced_images/xray_enhanced/000068 ...,/content/enhanced_images/pres_enhanced/1500.png
1,P1001,Obesity,/content/enhanced_images/xray_enhanced/000077 ...,/content/enhanced_images/pres_enhanced/2929.png
2,P1002,Obesity,/content/enhanced_images/xray_enhanced/000162 ...,/content/enhanced_images/pres_enhanced/732.png
3,P1003,Diabetes,/content/enhanced_images/xray_enhanced/18.png,/content/enhanced_images/pres_enhanced/277.png
4,P1004,Cancer,/content/enhanced_images/xray_enhanced/000078 ...,/content/enhanced_images/pres_enhanced/675.png


💾 Saved merged dataset → /content/cleaned_data/healthcare_with_images.csv


# Summary of Work

In this notebook, we successfully:
- Loaded and preprocessed the healthcare dataset, including cleaning text, formatting dates and age, and standardizing admission types.
- Added a unique patient ID to the dataset.
- Loaded and cleaned the ICD-10 codes dataset and the enhanced image index.
- Performed a quick, random assignment of ICD codes to patient records.
- Linked synthetic X-ray and prescription image paths to each patient record.
- Saved the intermediate and final preprocessed datasets.

This integrated dataset is now ready to be used for generating clinical notes or other text-based AI processing tasks.